In [1]:
import pandas as pd
DelDF = pd.read_csv('Deliveries.csv')
OldDelDF = pd.read_csv('DeliveriesOld.csv')
DaysDF = pd.read_csv('Days.csv')
WeeksDF = pd.read_csv('Weeks.csv')
gpsDF = pd.read_csv('GPS.csv')
rawDF = pd.read_csv('RawData.csv')

In [2]:
def dollars_to_floats(df, columns):
    for x in columns:
        df[x] = df[x].str.replace('$', '')
        df[x] = df[x].str.replace(',', '')
        df[x] = df[x].astype(float)

In [3]:
frames = [OldDelDF, DelDF]
result = pd.concat(frames)
#We need to reset the index after the combination of v1/v2, and v3
result.reset_index(drop=True, inplace=True)

#DelDF is v3
#results is all deliveries
#Those are the only 2 we need

In [4]:
result.dtypes

ID                  int64
Date               object
Restaurant Name    object
Total              object
Pay                object
Tip                object
Peak Bonus         object
Stacked            object
Version             int64
Start Time         object
Miles              object
End Time           object
Est. Duration      object
dtype: object

In [5]:
dollars_to_floats(result, ["Total", "Pay", "Tip", "Peak Bonus"])
dollars_to_floats(OldDelDF, ["Total", "Pay", "Tip", "Peak Bonus"])

/tmp/ipykernel_43337/4254552812.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[x] = df[x].str.replace('$', '')


In [6]:
#Doodle to get the percentage of orders that tip
numUntipped = (result["Tip"] == 0.0).sum()
numTipped = (result["Tip"] != 0.0).sum()
percTipped = (numTipped/(numUntipped+numTipped))
print(percTipped*100, "% of customers tip")

89.2663043478261 % of customers tip


In [7]:
#Doodle that converts 'Date' attribute to Timestamp object

from datetime import date, time

datevar = result.iloc[600]["Date"]
timevar = result.iloc[600]["Start Time"]
print(datevar, " ", timevar)
datevar1 = '2023/' + datevar + " " + timevar
datevar1 = pd.Timestamp(datevar1)
print(datevar1)
print(datevar1.day_name())
print(type(datevar1))

5/27   7:19:00 PM
2023-05-27 19:19:00
Saturday
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [8]:
#Only convert v3 to datetime, except Date which can all be converted
datetest = result.iloc[600]["Date"] + "/2023"
timetest = result.iloc[600]["Start Time"]
datetest = pd.to_datetime(datetest, format="%m/%d/%Y")
print(type(datetest)) #this is a Timestamp and note datetime object?
print(datetest.strftime('%m-%d'), timetest)

#So maybe what we want to do, is convert date, start, end to datetime, exclude v1/v2 for start/end related analysis
#now try convert start time to date time
timetest1 = pd.to_datetime(timetest)
print(timetest1)

#interesting, it automatically uses the current date without an existing date
#maybe we just convert both to datetime, then use datetime.date for one and datetime.time for the other

print(timetest1.hour, timetest1.minute, datetest.month, datetest.day)
datetimetest = pd.Timestamp.combine(date(datetest.year, datetest.month, datetest.day), time(timetest1.hour, timetest1.minute))
print(datetimetest)
print(type(datetimetest))

#Okay so that's how we can get the timestamp for date and for time, that leaves miles and duration to check

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
05-27 7:19:00 PM
2023-06-21 19:19:00
19 19 5 27
2023-05-27 19:19:00
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [9]:
# GPS DOODLE HERE
# Initial doodle to help with address entry
# Need to rework to handle both CSVs now
#gpsDF1 = gpsDF[["ID", "Date", "Restaurant Name", "RAW DATA"]].copy()
#gpsDF2 = gpsDF1[["Restaurant Name", "RAW DATA"]].copy()
#gpsDF2 = gpsDF2.drop_duplicates(subset=["Restaurant Name", "RAW DATA"])
#gpsDF2 = gpsDF2.sort_values(by="Restaurant Name", ascending=True)
#gpsDF2.to_csv('gpsDF2.csv')

In [10]:
#Why not try
#result[result["Peak Bonus"] != 0 ].shape

tempDF = result[result["Peak Bonus"] != 0 ]
tempDF.shape
#Since a significant volume of orders are off peak
#(because peak happens about 1am-3am)
#And since I dont really want to spend lots of time working those hours
#It might make sense to re-evaluate the pursuit of peak pay based insights

(45, 13)

In [11]:
#tempDF = result[result["Restaurant Name"].duplicated(keep=False) == False]
#This is all the places I've picked up from only once
#duplicated(keep=False) flags all rows that appear more than once to be flagged as duplicates instead of default
#default is all rows after the first flagged as duplicates

In [12]:
tempDF = result[result["Restaurant Name"].duplicated(keep=False) == True]
tempDF = tempDF.groupby(["Restaurant Name"]).agg({"Pay": "mean", "Tip": "mean"})
tempDF = tempDF.round({'Pay': 2, 'Tip': 2})
tempDF.sort_values("Tip", ascending=False)

,Pay,Tip
Restaurant Name,,
Albertsons,4.58,12.39
Hunter Steakhouse,10.00,11.50
Chin's Szechwan,5.88,9.25
Mangia e Bevi,6.25,9.00
Maans Mediterranean Grill and Kabobery,5.75,7.70
...,...,...
Lollicup Fresh,4.25,1.82
Carl's Jr.,4.38,1.50
Sizzler,12.75,1.25


In [13]:
#So we see car parts and flower orders as the only actual 0 tip
#On the other hand, grocery orders tip well, which makes sense because they are much more work than normal
#And we see large avg tip values from two pizza places in the top 5
#I think it would be more useful to have a count as a 4th column

In [14]:
#Overall hourly rate, Total from TOTALS / Dash
#Hardcoded index for Totals, should look for row where dates == "TOTALS" 
print(WeeksDF.shape)
totalPay = WeeksDF.loc[12]["Total"]
totalPay = totalPay.replace('$', '')
totalPay = totalPay.replace(',', '')
totalPay = float(totalPay)

totalTime = WeeksDF.loc[12]["Dash"]
totalTimeH = totalTime[0:3]
totalTimeM = totalTime[4:6]
#okay so it includes the first position and does not include the second position, so its a [x, y) situation
totalTimeH = float(totalTimeH)
totalTimeM = float(totalTimeM)
totalTimeM = totalTimeM/60
totalTime = totalTimeH + totalTimeM

hourlyRate = totalPay/totalTime
print("$", hourlyRate, "per hour.")

(13, 11)
$ 22.20477677177646 per hour.


In [15]:
#So float(var) converts, but series.astype(float) is for series in df

In [16]:
#Doodle to 
#convert $#.## to float and hh:mm:ss to float, 
#then make the pre total to hours worked ratio, then sort and show
#Should change this to use function dollars_to_floats

print(WeeksDF.dtypes)

WeeksDF["Pre Total"] = WeeksDF["Pre Total"].str.replace('$', '')
WeeksDF["Pre Total"] = WeeksDF["Pre Total"].str.replace(',', '')
WeeksDF["Pre Total"] = WeeksDF["Pre Total"].astype(float)

#print(type(active))

for index, row in WeeksDF.iterrows():
    active = row["Active"]
    #print(active)
    #print(type(active))
    activeH = float(active[:active.index(":")])
    activeM = round(float(active[active.index(":")+1:active.index(":")+3])/60, 2)
    active = activeH + activeM
    WeeksDF.at[index, "Active"] = active
#Okay now we have pre total and active in a format that allows for good math
WeeksDF["pretotalactiveratio"] = WeeksDF["Pre Total"]/WeeksDF["Active"]
WeeksDF.sort_values(by=["pretotalactiveratio"])
#right well its kind of interesting to see totals in the middle
#and it also looks like May had better ratios, which makes it seem like I'm getting better?
#better isn't a good characterization
#pre total is the money before adjustment, its just base pay + tip 
#so may having better base pay + tip / time actually spent on deliveries is interesting

Dates         object
Base          object
Tip           object
Adj pay       object
Other         object
Pre Total     object
Total         object
Active        object
Dash          object
Deliveries     int64
Version       object
dtype: object


/tmp/ipykernel_43337/3250166319.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  WeeksDF["Pre Total"] = WeeksDF["Pre Total"].str.replace('$', '')


,Dates,Base,Tip,Adj pay,Other,Pre Total,Total,Active,Dash,Deliveries,Version,pretotalactiveratio
2,4/10 - 4/16,$130.75,$96.13,$61.07,$0.00,226.88,$287.95,8.63,11:10:00,28,1,26.289687
4,4/24 - 4/30,$430.00,$299.73,$138.76,$2.62,729.73,$871.11,25.5,41:10:00,84,1,28.616863
12,TOTALS:,"$3,544.75","$2,714.44",$844.13,$8.87,6259.19,"$7,112.19",216.62,320:18:00,674,-,28.894793
0,3/27 - 4/2,$108.50,$88.00,$40.96,$0.00,196.50,$237.46,6.7,10:55:00,22,1,29.328358
3,4/17 - 4/23,$382.50,$248.41,$105.05,$0.00,630.91,$735.96,21.33,32:18:00,68,1,29.578528
7,5/15 - 5/21,$303.75,$239.25,$99.27,$0.00,543.00,$642.27,17.53,25:24:00,64,2,30.975471
10,6/5 - 6/11,$197.25,$185.09,$65.38,$0.00,382.34,$447.72,11.77,15:45:00,41,3,32.484282
6,5/8 - 5/14,$758.25,$567.84,$155.48,$6.25,1326.09,"$1,487.82",40.58,58:08:00,138,2,32.678413
5,5/1 - 5/7,$618.50,$445.32,$121.74,$0.00,1063.82,"$1,185.56",32.4,50:40:00,114,1,32.833951
9,5/29 - 6/4,$342.75,$309.88,$115.30,$0.00,652.63,$767.93,19.4,26:32:00,63,3,33.640722


In [17]:
#Dollars Earned vs Miles Driven doodle
#$ to float
#drop non numeric Miles values, then convert column to float
#agg sum Total and Miles, by restaurant name (then round)
#create new column as Total/Miles
#sort low to high
v3DF = DelDF.copy()
v3DF["Total"] = v3DF["Total"].str.replace('$', '')
v3DF["Total"] = v3DF["Total"].str.replace(',', '')
v3DF["Total"] = v3DF["Total"].astype(float)
v3DF = v3DF[v3DF.Miles!="-"]
v3DF["Miles"] = v3DF["Miles"].astype(float)
v3DF = v3DF.groupby(["Restaurant Name"]).agg({"Total": "sum", "Miles": "sum"})
v3DF = v3DF.round({'Pay': 2, 'Tip': 2})
v3DF["DollarMilesRatio"] = v3DF["Total"]/v3DF["Miles"]
v3DF.sort_values(by="DollarMilesRatio")

/tmp/ipykernel_43337/2746804118.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  v3DF["Total"] = v3DF["Total"].str.replace('$', '')


,Total,Miles,DollarMilesRatio
Restaurant Name,,,
Pedro's Tacos,7.00,7.9,0.886076
Anita's Mexican Restaurant & Cantina,4.75,4.9,0.969388
Urban Plates,21.50,20.4,1.053922
Lollicup Fresh,6.25,5.6,1.116071
Colimas Mexican Food,6.25,5.3,1.179245
...,...,...,...
Cafe de Thai & Sushi,21.75,5.7,3.815789
Albertsons,34.17,8.8,3.882955
Graziano's Piza,9.50,2.3,4.130435


In [18]:
DaysDF.max() #would be useful with more attributes converted to float 

Date                    6/9
Total                $86.13
Base                 $99.75
Tip                  $94.03
Start (24)            18:58
End (24)               2:32
Active time (h:m)      9:40
Dash time (h:m)        9:41
Offers                   32
Deliveries               33
Mileage Start         69045
Mileage End           69192
Version                   3
Pauses                    2
dtype: object

In [19]:
#Avg tip vs Avg tip excluding zero tip orders

TipAvg = result["Tip"].mean()
print(TipAvg)
TipAvg2 = result[result["Tip"] !=0]["Tip"].mean()
print(TipAvg2)

4.076875
4.567092846270929


In [20]:
#doodle checking if dollars_to_floats works
tempDF = DelDF.copy()
listToPass = ["Total", "Pay", "Tip", "Peak Bonus"]
dollars_to_floats(tempDF, listToPass)
#Wanted to check Miles to see which non numeric values we had to drop
tempDF['Miles'].unique()

/tmp/ipykernel_43337/4254552812.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[x] = df[x].str.replace('$', '')


array(['6.2', '2.9', '5.5', '3.1', '3.3', '7.1', '4.84', '5.2', '3.7',
       '4.7', '3.0', '5.9', '5.1', '-', '8.2', '6.1', '7.2', '3.2', '2.6',
       '2.4', '9.5', '8.3', '2.3', '6.9', '4.5', '7.7', '2.5', '5.0',
       '1.8', '4.0', '7.3', '6.3', '5.3', '7.5', '2.0', '3.4', '3.6',
       '4.2', '6.0', '2.7', '7.4', '10.1', '8.0', '15.9', '7.6', '6.5',
       '4.4', '1.9', '4.3', '4.1', '1.4', '4.8', '1.2', '6.8', '10.2',
       '12.1', '1.3', '0.8', '6.4', '6.7', '4.6', '1.6', '5.6', '2.2',
       '10.8', '4.9', '3.5', '5.7', '1.7', '7.9'], dtype=object)

In [21]:
#tempDF doodle
#Drop the non numeric values (which are just '-') and convert Miles to float from string
tempDF = tempDF[tempDF['Miles'] != '-']
tempDF['Miles'] = tempDF['Miles'].astype(float)

In [22]:
#DATETIME DOODLE

date = tempDF.iloc[-1]["Date"]
starttime = tempDF.iloc[-1]["Start Time"]
endtime = tempDF.iloc[-1]["End Time"]
print(date, starttime, endtime)
date = '2023/' + datevar + " " + starttime
date = pd.Timestamp(date)
starttime = date
endtime = '2023/' + datevar + " " + endtime
endtime = pd.Timestamp(endtime)
print("\n", date)
print(date.day_name())
print(starttime.minute)
print(endtime.minute)

6/18 8:26:00 PM 9:05:00 PM

 2023-05-27 20:26:00
Saturday
26
5


In [23]:
#datetest = result.iloc[600]["Date"] + "/2023"
#timetest = result.iloc[600]["Start Time"]
#datetest = pd.Timestamp(datetest)
#datetest = pd.to_datetime(datetest, format="%m/%d/%Y")
#print(datetest.strftime('%m-%d'), timetest)


#timetest1 = pd.to_datetime(timetest)

#datetimetest = pd.Timestamp.combine(date(datetest), time(timetest1))
#print(timetest1.hour, timetest1.minute, datetest.month, datetest.day)
#datetimetest = pd.Timestamp.combine(date(datetest.year, datetest.month, datetest.day), time(timetest1.hour, timetest1.minute))
#print(datetimetest)

In [24]:
#Doodle of loop that adds 2023 to the date string to allow for easier conversion to datetime object
for index, row in DaysDF.iterrows():
    date = pd.Timestamp('2023/' + row["Date"])
    DaysDF.at[index, "Date"] = date
    #the below should work instead of the line outside the loop
    #DaysDF.at[index, "Date"] = pd.to_datetime(date)
    #or = pd.Timestamp(date)
#Converts date from string to datetime
DaysDF["Date"] = pd.to_datetime(DaysDF["Date"])

In [25]:
#Adds column to DaysDF for the day of the week (Monday, Thursday, etc.)
DaysDF["Day of Week"] = DaysDF["Date"].dt.day_name()

In [26]:
#Doodle of for loop and type cast to convert from hh:mm to float
#This should be made into a function
for index, row in DaysDF.iterrows():
    #print(row["Active"])
    active = row["Dash time (h:m)"]
    activeH = float(active[:active.index(":")])
    activeM = round(float(active[active.index(":")+1:active.index(":")+3])/60, 2)
    active = activeH + activeM
    #print(active)
    DaysDF.at[index, "Dash time (h:m)"] = active
#Converts the string value for hours worked into float
DaysDF["Dash time (h:m)"] = DaysDF["Dash time (h:m)"].astype(float)

In [27]:
#Converts the string value for hours worked into float
DaysDF["Dash time (h:m)"] = DaysDF["Dash time (h:m)"].astype(float)
#Converts Total, Base, and Tip attributes to float using the dollars_to_floats func
dollars_to_floats(DaysDF, ["Total", "Base", "Tip"])
#Gets the total amount of hours worked, total pay, total tips earned, and total deliveries by day of week
DaysDF = DaysDF.groupby(["Day of Week"]).agg(
    {"Dash time (h:m)": "sum", "Total": "sum", "Tip": "sum", "Deliveries ": "sum"})
DaysDF = DaysDF.round({'Dash time (h:m)': 2, 'Total': 2, "Tip": 2})
#Creates a new column to get the hourly pay rate by day of week
DaysDF["$/hr"] = DaysDF["Total"]/DaysDF["Dash time (h:m)"]
#Creates a new column to get the average tip per order by day of the week
DaysDF["AvgTip"] = DaysDF["Tip"]/DaysDF["Deliveries "]
display(DaysDF)

/tmp/ipykernel_43337/4254552812.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[x] = df[x].str.replace('$', '')


,Dash time (h:m),Total,Tip,Deliveries,$/hr,AvgTip
Day of Week,,,,,,
Friday,94.85,2082.03,940.78,221,21.950764,4.256923
Monday,7.02,135.97,56.97,14,19.368946,4.069286
Saturday,75.89,1744.22,782.72,178,22.983529,4.397303
Sunday,37.63,882.78,408.03,90,23.459474,4.533667
Thursday,56.86,1167.71,484.21,134,20.536581,3.613507
Wednesday,48.00,845.87,327.87,99,17.622292,3.311818
